In [56]:
import numpy as np
from bs4 import BeautifulSoup
import requests
import pymysql 
from datetime import date
from datetime import datetime

In [57]:
#Empezamos el scraping
#Obtener el HTML
URL_BASE = 'https://cuspide.com/100-mas-vendidos/'
pedido_obtenido = requests.get(URL_BASE)
html_obtenido = pedido_obtenido.text

#"Parsear" ese HTML
soup = BeautifulSoup(html_obtenido, "html.parser")

In [58]:
#sirve para sacar nombre y url
h3_todos = soup.find_all('h3')
list(h3_todos)
len(h3_todos)

#transformo el contenido de las lista en str, para usar metodos de str
for i,l in enumerate(h3_todos):
    h3_todos[i] = str(l) 

In [59]:
#Se puede sacar el precio, despues del indice 0
bdi_todos = soup.find_all('bdi')
list(bdi_todos)
len(bdi_todos)

#elinimo el primer elemento de la lista
bdi_todos.pop(0)

#transformo el contenido de las lista en str, para usar metodos de str
for i,l in enumerate(bdi_todos):
    bdi_todos[i] = str(l) 

In [60]:
#separa los str para sacar la informacion
nom_url=[]
for i,l in enumerate(h3_todos):
    nom_url.append(h3_todos[i][l.find('href=')+5 :])

In [61]:
#URL
#saca la url de la lista de string
url_=[]
for i,l in enumerate(nom_url):
    url_.append(nom_url[i][ :l.find('>')])

#quita la comilla doble del string del url
for i,l in enumerate(url_):    
    url_[i] = l.replace('"','')

In [62]:
#Consigo los nombres de los libros
nombre=[]
for i,l in enumerate(nom_url):
    nombre.append(nom_url[i][l.find('>')+1 : l.find('<')])

In [63]:
#se tiene el valor de los precios, pero en notacion latina 0.000,00
precio=[]
for i,l in enumerate(bdi_todos):
    precio.append(bdi_todos[i][l.find('\xa0')+1 : l.rfind('<')])

#paso el precion a notacion internacional y transformo en float
for i,l in enumerate(precio):
     precio[i] = float(l.replace('.','').replace(',','.'))    

In [64]:
#scraping para la transformacion de pesos argentinos a dolares
#Obtener el HTML
URL_BASE1 = 'https://www.xe.com/es/currencyconverter/convert/?Amount=1&From=ARS&To=USD'
pedido_obtenido1 = requests.get(URL_BASE1)
html_obtenido1 = pedido_obtenido1.text

#"Parsear" ese HTML
soup1 = BeautifulSoup(html_obtenido1, "html.parser")

#sirve para sacar nombre y url
body = soup1.find_all('body')
list(body)
body[0] = str(body[0])


#toma la parte decimal 0.00
indice = int(body[0].find('"result__BigRate-sc-1bsijpp-1 iGrAod"'))

a = body[0][indice + 38 : indice + 42]
#toma los decimaneles luego del 0.00
b = body[0][indice + 69: indice + 74]
#viene en formato latino, toca pasarlo a formato de usa
a = a.replace(',','.')

#concadena los calores y los vuelve un string
ars_usd = float(a+b)

In [65]:
#scraping para la transformacion de dolares a euros
#Obtener el HTML
URL_BASE1 = 'https://www.xe.com/es/currencyconverter/convert/?Amount=1&From=USD&To=EUR'
pedido_obtenido1 = requests.get(URL_BASE1)
html_obtenido1 = pedido_obtenido1.text

#"Parsear" ese HTML
soup1 = BeautifulSoup(html_obtenido1, "html.parser")

#sirve para sacar nombre y url
body = soup1.find_all('body')
list(body)
body[0] = str(body[0])

#toma la parte decimal 0.00
indice = int(body[0].find('"result__BigRate-sc-1bsijpp-1 iGrAod"'))

a = body[0][indice + 38 : indice + 42]
#toma los decimaneles luego del 0.00
b = body[0][indice + 69: indice + 75]
#viene en formato latino, toca pasarlo a formato de usa
a = a.replace(',','.')

#concadena los calores y los vuelve un string
usd_eur = float(a+b)

In [66]:
#Agregar columna de precio en dolares y euros
#toca trabajar como arreglo de numpy
precio = np.array(precio).round(2)
precio_dolar = (precio*ars_usd).round(2)
precio_euro = (precio_dolar*usd_eur).round(2)

#transforma los arreglos a lista
precio = list(precio)
precio_dolar = list(precio_dolar)
precio_euro = list(precio_euro)


In [67]:
#genera una lista con la fecha
now = datetime.now()
fecha=[]
fecha1=[]
for i in nombre:
 fecha.append(now.strftime("%Y-%m-%d"))


In [68]:
#transforma las listas con los datos a una lista de tuplas
pre_lista=[]
datos=[]

for i in range(0,len(nombre)): 
    pre_lista = [nombre[i], url_[i], precio[i], precio_dolar[i], precio_euro[i], fecha[i]]
    datos.append(tuple(pre_lista))

In [69]:
#Establece la conexion con el servidor local, sin especificar la base de datos
conexion = pymysql.connect( host='localhost', user='root', passwd='6206')
#crea cursor
cursor = conexion.cursor()

In [70]:
#crea la base de datos
crear_base = """ CREATE DATABASE IF NOT EXISTS top_libros; """
cursor.execute(crear_base)

1

In [71]:
#cierro y vuelvo a abrir la concexion ya dentro de la base de datos creada
#si no lo hago da un error, no me sirvio usar USE oferta_gastronomica;

conexion.close()
#Establece la conexion con el servidor local, ya dento de base de datos
conexion = pymysql.connect( host='localhost', user='root', passwd='6206', db='top_libros' ) 
#crea cursor
cursor = conexion.cursor()

In [72]:
#Crear tabla
crear_tabla = """
    CREATE TABLE  top (
	idLibro INT NOT NULL AUTO_INCREMENT,
    titulo VARCHAR (60),
    url VARCHAR (120),
    precio FLOAT (2),
    precio_dolar FLOAT (2),
    precio_euro FLOAT (2),
    fecha DATE,
    PRIMARY KEY (idLibro)
    );"""
cursor.execute(crear_tabla)


0

In [73]:
#Crear tabla de errores
# crear_tabla = """
#     CREATE TABLE  errores (
# 	idError INT NOT NULL AUTO_INCREMENT,
#     titulo VARCHAR (60),
#     url VARCHAR (120),
#     fecha DATE,
#     PRIMARY KEY (idError)
#     );"""
# cursor.execute(crear_tabla)

0

In [74]:
#insertar valores
cursor.executemany("INSERT INTO top (titulo, url, precio, precio_dolar, precio_euro, fecha) VALUES (%s, %s, %s, %s, %s, %s)", datos)

91

In [75]:
#para insertar los datos en la tabla
conexion.commit()
#cierra conexion
conexion.close()